In [ ]:
# 필요한 패키지
from selenium import webdriver  
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from urllib import request as urlreq
import time
import re
from tqdm import tqdm_notebook
from openpyxl import load_workbook 
import openpyxl as xl


# 키워드, 기간설정, 페이지 번호에 따른 url 가져오는 함수 생성
def mk_url(keyword, from_date, to_date, page_num):
    
    #최신순으로 정렬한 path
    path = 'https://kin.naver.com/search/list.naver?sort=date&query='
    
    period = '&period=' + from_date + '.%7C' + to_date
    url = path + keyword + period + '.&section=qna&page=' + page_num
    
    return url

# 기간설정을 위한 날짜 만드는 함수 생성
def mk_date(month, day):
    f = '2022.%s.%s'%(month.zfill(2), day.zfill(2))
    t = '2022.%s.%s'%(month.zfill(2), day.zfill(2))
    return f, t


############키워드, 날짜 설정하기 위한 변수 설정 부분############
# 한달마다 기간설정하여 크롤링 진행
keyword = '우울증'
month = 3
s_day = 1
e_day = 31
################################################################


date_info = str(month).zfill(2) + str(s_day).zfill(2) + '_' + str(month).zfill(2) + str(e_day).zfill(2) 
f_name = keyword + '_' + date_info +"_crawling_result.xlsx"

#엑셀로 저장하기 위한 변수 설정
wb = xl.Workbook()
sheet = wb.active
sheet.append(['date','user_name','title','content','tag'])


# 크롤링 수행
for i in tqdm_notebook(range(s_day, e_day + 1), desc = '페이지 수집중...'):
    f, t = mk_date(str(month), str(i))
    post_number = None
    current_number = None
    total_number = None
    page_num = 1
    page_url = []
    
    # 1차 크롤링 - 페이지별로 게시글 url 수집
    while True:
        #최신순으로 크롤링
        
        # 크롬 드라이버
        binary = "C:\\data\\chromedriver_win32\\chromedriver.exe"

        # 브라우저를 인스턴스화
        driver = webdriver.Chrome(binary)

        kwd_url = mk_url(keyword,f,t,str(page_num))
        driver.get(kwd_url)
        time.sleep(0.5)
        html = driver.page_source

        soup = BeautifulSoup(html,'html.parser')
        base = soup.find_all('a',class_="_nclicks:qna.txt _searchListTitleAnchor")

        # 개별 주소 가져오기
        for b_num in base:
            page_url.append(b_num.get('href')) 

        # 페이지 넘기기
        post_number = driver.find_element_by_class_name('number').text
        
        current_number = post_number[post_number.find('-')+1:post_number.find('/')]
        current_number = re.sub(',','', current_number)
        
        total_number = post_number[post_number.find('/')+1:post_number.find(')')]
        total_number = re.sub(',','', total_number)
        
        if int(current_number) == int(total_number):
            break
            
        else:
            page_num += 1
            time.sleep(0.5)
        
    page_url = list(set(page_url))  # 페이지 url 중복제거

    # 2차 크롤링 - 각 페이지 게시내용 크롤링
    for i in tqdm_notebook(page_url, desc = '게시글 수집중....'):
        try:
            driver.get(i)
            time.sleep(0.5)
            
            #upload_date
            upload_date = driver.find_element_by_class_name("c-userinfo__info").text
            upload_date = upload_date[upload_date.find('\n')+1:]

            #user name
            user_name = driver.find_element_by_class_name("c-userinfo__author").text
            user_name = user_name[user_name.find('\n')+1:]

            #제목
            title = driver.find_element_by_class_name('title').text

            #내용
            try:
                question_txt = driver.find_element_by_class_name('c-heading__content').text
            except:
                question_txt = ""

            question_txt = re.sub('[\n\r\t]+','',question_txt)

            #태그
            tag = driver.find_element_by_class_name("tag-list__item.tag-list__item--category").text
            tag = tag[tag.find('Ξ')+2:]

            # 엑셀에 게시날짜, 제목, 내용, 태그 저장
            sheet.append([upload_date, user_name, title, question_txt, tag])
            time.sleep(0.5)

        except:
            continue
    

    driver.quit()
    
wb.save("c:\\naver_qna\\" + f_name)  # 엑셀로 저장
